In [1]:
import cv2
import cv2.aruco as aruco
import numpy as np

cap = cv2.VideoCapture('CharUco_board.mp4')
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) // 2
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) // 2

arucoParams = aruco.DetectorParameters()
arucoParams.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX
arucoDict = aruco.getPredefinedDictionary(aruco.DICT_6X6_250)
detector = aruco.ArucoDetector(arucoDict, arucoParams)

gridX, gridY = 5, 7
squareSize = 4
charucoBoard = aruco.CharucoBoard((gridX, gridY), squareSize, squareSize/2, arucoDict)

cameraMatrixInit = np.array([[frameWidth / 2., 0., frameWidth / 2.],
                             [0., frameWidth / 2., frameHeight / 2.],
                             [0., 0., 1.]])
distCoeffsInit = np.zeros((5, 1))

charucoParams = aruco.CharucoParameters()
charucoParams.tryRefineMarkers = True
charucoParams.cameraMatrix = cameraMatrixInit
charucoParams.distCoeffs = distCoeffsInit
charucoDetector = aruco.CharucoDetector(charucoBoard, charucoParams, arucoParams)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-5)
frameId = 0
objPointsList = []
imgPointsList = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (frameWidth, frameHeight))
    corners, ids, _, _ = charucoDetector.detectBoard(frame)

    if corners is not None and corners.shape[0] > 0:
        objPoints, imgPoints = charucoBoard.matchImagePoints(corners, ids)
        if frameId % 100 == 50 and objPoints.shape[0] >= 4:
            objPointsList.append(objPoints)
            imgPointsList.append(imgPoints)
    frameId += 1

cap.release()

ret, cameraMatrix, distCoeffs, _, _ = cv2.calibrateCamera(
    objPointsList, imgPointsList,
    (frameWidth, frameHeight),
    cameraMatrixInit, distCoeffsInit,
    flags=cv2.CALIB_USE_INTRINSIC_GUESS,
    criteria=criteria
)

print("cameraMatrix =\n", cameraMatrix)
print("distCoeffs =\n", distCoeffs)

cameraMatrix =
 [[879.94310174   0.         481.92858403]
 [  0.         888.62969728 311.59075656]
 [  0.           0.           1.        ]]
distCoeffs =
 [[ 0.0735621 ]
 [-0.3277629 ]
 [-0.00159971]
 [-0.0026317 ]
 [ 0.86872408]]


In [6]:
cap = cv2.VideoCapture('arUco_marker.mp4')
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) // 2
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) // 2
fps = cap.get(cv2.CAP_PROP_FPS)

marker_video_paths = {
    1: 'vid_0.mp4',
    2: 'vid_1.mp4',
    3: 'vid_2.mp4',
    4: 'vid_3.mp4',
    5: 'vid_4.mp4',
    6: 'vid_5.mp4'
}

overlay_caps = {}
for marker_id, path in marker_video_paths.items():
    cap_overlay = cv2.VideoCapture(path)
    overlay_caps[marker_id] = cap_overlay


fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (frameWidth, frameHeight))

arucoDict = aruco.getPredefinedDictionary(aruco.DICT_7X7_50)
arucoParams = aruco.DetectorParameters()
arucoParams.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX
detector = aruco.ArucoDetector(arucoDict, arucoParams)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, (frameWidth, frameHeight))
    
    corners, ids, _ = detector.detectMarkers(frame)
    if ids is not None:
        ids = ids.flatten()
        for i, marker_id in enumerate(ids):
            if marker_id not in overlay_caps:
                continue

            overlay_cap = overlay_caps[marker_id]
            ret_overlay, overlay_frame = overlay_cap.read()
            if not ret_overlay:
                overlay_cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                ret_overlay, overlay_frame = overlay_cap.read()
            if not ret_overlay:
                continue

            overlay_frame = cv2.resize(overlay_frame, (120, 120))
            overlay_frame = cv2.flip(overlay_frame, 0)

            pts_dst = corners[i][0].astype(np.float32)
            h, w = overlay_frame.shape[:2]
            pts_src = np.array([[0, 0], [w - 1, 0], [w - 1, h - 1], [0, h - 1]], dtype=np.float32)

            H, _ = cv2.findHomography(pts_src, pts_dst)
            warped = cv2.warpPerspective(overlay_frame, H, (frame.shape[1], frame.shape[0]))

            mask = np.zeros_like(frame, dtype=np.uint8)
            cv2.fillConvexPoly(mask, pts_dst.astype(int), (255, 255, 255))
            mask_inv = cv2.bitwise_not(mask)
            frame_bg = cv2.bitwise_and(frame, mask_inv)
            warped_fg = cv2.bitwise_and(warped, mask)

            frame = cv2.add(frame_bg, warped_fg)

    out.write(frame)

cap.release()
out.release()
for cap_overlay in overlay_caps.values():
    cap_overlay.release()